In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../../data/project_scores.csv')

In [3]:
df

,user_id,project_id,score
0,5f5a90e2-6601-480c-bc08-7592e06beacd,45ba1903-cafe-4e57-b7ec-1de402d32cc1,1
1,5f5a90e2-6601-480c-bc08-7592e06beacd,22a9acc5-428a-41d4-b056-9bebe9183afe,1
2,0db6b893-14aa-4ae5-8e57-692ca7943ab5,37e5167f-10d9-429d-99c4-260d68e39307,2
3,0db6b893-14aa-4ae5-8e57-692ca7943ab5,467f58b6-c640-493a-aceb-36dea9881778,2
4,5f5a90e2-6601-480c-bc08-7592e06beacd,6f6235eb-a751-43f2-8bad-506c884635db,4
5,0db6b893-14aa-4ae5-8e57-692ca7943ab5,41edbc3e-2636-4806-bd87-4c19f273bb3c,4
6,0db6b893-14aa-4ae5-8e57-692ca7943ab5,45ba1903-cafe-4e57-b7ec-1de402d32cc1,4
7,0db6b893-14aa-4ae5-8e57-692ca7943ab5,6f6235eb-a751-43f2-8bad-506c884635db,4
8,5f5a90e2-6601-480c-bc08-7592e06beacd,41edbc3e-2636-4806-bd87-4c19f273bb3c,4


In [4]:
from sklearn.preprocessing import LabelEncoder

user_le = LabelEncoder()
df['enc_user_id'] = user_le.fit_transform(df['user_id'])

project_le = LabelEncoder()
df['enc_project_id'] = project_le.fit_transform(df['project_id'])

In [5]:
X = df[['enc_user_id', 'enc_project_id']]
y = df['score']

In [6]:
import tensorflow as tf
from keras.models import Model
from keras.layers import Input,Dense,Embedding,Flatten,Input,concatenate,Add
from keras.regularizers import l2

In [7]:
num_users = df['enc_user_id'].nunique()
num_projects = df['enc_project_id'].nunique()
k = 50
l2_lambda = 0.001

In [8]:
u_input = Input((1,), name='user_input')
u = Embedding(num_users, k, name='user_emb')(u_input)
u = Flatten(name='user_flat')(u)
u = Dense(48, activation='relu', name='user_dense')(u)

In [9]:
p_input = Input((1,), name='project_input')
p = Embedding(num_projects, k, name='project_emb')(p_input)
p = Flatten(name='project_flat')(p)
p = Dense(8, activation='relu', name='project_dense')(p)

In [10]:
x = concatenate([u, p], name='concat')
# x = Dropout(0.1, name='drop1')(x)
x = Dense(16, activation='relu', name='dense1')(x)
x = Dense(4, activation='relu', name='dense2')(x)

In [11]:
u_bias = Embedding(num_users, 1, embeddings_regularizer=l2(l2_lambda), name='user_bias_emb')(u_input)
u_bias = Flatten(name='user_bias_flat')(u_bias)

p_bias = Embedding(num_projects, 1, embeddings_regularizer=l2(l2_lambda), name='project_bias_emb')(p_input)
p_bias = Flatten(name='project_bias_flat')(p_bias)

In [12]:
o = concatenate([x, u_bias, p_bias], name='combined_features')
o = Dense(16, activation='relu', name='combined_dense1')(o)
o = Dense(4, activation='relu', name='combined_dense2')(o)
o = Dense(1, activation='linear' , name='output')(o)

In [13]:
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsoluteError, RootMeanSquaredError

model = Model(inputs=[u_input, p_input], outputs=o)

metrics = [
    MeanSquaredError(name='mse'),
    MeanAbsoluteError(name='mae'),
    RootMeanSquaredError(name='rmse')
]

model.compile(optimizer=tf.keras.optimizers.legacy.Adam(0.001), loss='mean_squared_error', metrics = metrics)

In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user_input (InputLayer)     [(None, 1)]                  0         []                            
                                                                                                  
 project_input (InputLayer)  [(None, 1)]                  0         []                            
                                                                                                  
 user_emb (Embedding)        (None, 1, 50)                100       ['user_input[0][0]']          
                                                                                                  
 project_emb (Embedding)     (None, 1, 50)                300       ['project_input[0][0]']       
                                                                                              

In [15]:
from keras.utils import plot_model

plot_model(
    model,
    show_shapes=True,
    show_layer_names=True,
    expand_nested=True,
    show_layer_activations=True,
)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [16]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.1,
    patience=2,
    verbose=1,
    mode="min",
    baseline=None,
    restore_best_weights=True
)

In [17]:
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5',monitor='val_loss', verbose=1, save_best_only=True)

In [19]:
history = model.fit(
    x=[X['enc_user_id'], X['enc_project_id']],
    y=y, epochs=200,
    # validation_data=([X_val['userId'], X_val['movieId']], y_val),
    # callbacks=[early_stopping, model_checkpoint, model_weights_checkpoint]
)

Epoch 1/200
1/1 [==============================] - 0s 319ms/step - loss: 9.9757 - mse: 9.9757 - mae: 2.8855 - rmse: 3.1584
Epoch 2/200
1/1 [==============================] - 0s 2ms/step - loss: 9.9336 - mse: 9.9336 - mae: 2.8787 - rmse: 3.1518
Epoch 3/200
1/1 [==============================] - 0s 2ms/step - loss: 9.9065 - mse: 9.9065 - mae: 2.8742 - rmse: 3.1475
Epoch 4/200
1/1 [==============================] - 0s 2ms/step - loss: 9.8826 - mse: 9.8826 - mae: 2.8703 - rmse: 3.1437
Epoch 5/200
1/1 [==============================] - 0s 3ms/step - loss: 9.8585 - mse: 9.8585 - mae: 2.8661 - rmse: 3.1398
Epoch 6/200
1/1 [==============================] - 0s 3ms/step - loss: 9.8316 - mse: 9.8316 - mae: 2.8613 - rmse: 3.1355
Epoch 7/200
1/1 [==============================] - 0s 3ms/step - loss: 9.7989 - mse: 9.7989 - mae: 2.8554 - rmse: 3.1303
Epoch 8/200
1/1 [==============================] - 0s 3ms/step - loss: 9.7635 - mse: 9.7635 - mae: 2.8491 - rmse: 3.1247
Epoch 9/200
1/1 [=============

In [20]:
tf.keras.models.save_model(model, '../../models/projects/recommendations.h5')

/var/folders/8p/bh2hcnvn3bngf4_j4mcz1jx00000gn/T/ipykernel_979/663309186.py:1: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model, '../../models/projects/recommendations.h5')


In [32]:
user_embeddings = {}

counter = 0

for enc_user_id in df['enc_user_id'].unique():
    user_embedding = model.get_layer('user_emb')(np.array([enc_user_id]))
    user_embedding = tf.keras.backend.flatten(user_embedding)
    user_embedding = tf.expand_dims(user_embedding, axis=0)
    user_dense = model.get_layer('user_dense')(user_embedding)
    
    user_embeddings[user_le.inverse_transform([enc_user_id])[0]] = user_dense.numpy().tolist()

    counter+=1

    if counter%10000==0:
        print(f"Completed {counter} Iterations")

In [33]:
project_embeddings = {}

counter = 0

for enc_project_id in df['enc_project_id'].unique():
    project_embedding = model.get_layer('project_emb')(np.array([enc_project_id]))
    project_embedding = tf.keras.backend.flatten(project_embedding)
    project_embedding = tf.expand_dims(project_embedding, axis=0)
    project_dense = model.get_layer('project_dense')(project_embedding)
    
    project_embeddings[project_le.inverse_transform([enc_project_id])[0]] = project_dense.numpy().tolist()

    counter+=1

    if counter%10000==0:
        print(f"Completed {counter} Iterations")

In [34]:
user_bias_embeddings = {}

counter = 0

for enc_user_id in df['enc_user_id'].unique():
    user_bias_embedding = model.get_layer('user_bias_emb')(np.array([enc_user_id]))
    user_bias_embedding = tf.keras.backend.flatten(user_bias_embedding)

    user_bias_embeddings[user_le.inverse_transform([enc_user_id])[0]] = user_bias_embedding.numpy().tolist()

    counter+=1

    if counter%10000==0:
        print(f"Completed {counter} Iterations")

In [35]:
project_bias_embeddings = {}

counter = 0

for enc_project_id in df['enc_project_id'].unique():
    project_bias_embedding = model.get_layer('project_bias_emb')(np.array([enc_project_id]))
    project_bias_embedding = tf.keras.backend.flatten(project_bias_embedding)
    
    project_bias_embeddings[project_le.inverse_transform([enc_project_id])[0]] = project_bias_embedding.numpy().tolist()

    counter+=1

    if counter%10000==0:
        print(f"Completed {counter} Iterations")

In [40]:
import json

with open('../../models/projects/user_embeddings.json', 'w') as f:
    json.dump(user_embeddings, f)

with open('../../models/projects/project_embeddings.json', 'w') as f:
    json.dump(project_embeddings, f)

with open('../../models/projects/user_bias_embeddings.json', 'w') as f:
    json.dump(user_bias_embeddings, f)

with open('../../models/projects/project_bias_embeddings.json', 'w') as f:
    json.dump(project_bias_embeddings, f)

In [58]:
with open('../../models/projects/user_embeddings.json') as f:
    user_embeddings = json.load(f)
with open('../../models/projects/project_embeddings.json') as f:
    project_embeddings = json.load(f)
with open('../../models/projects/user_bias_embeddings.json') as f:
    user_bias_embeddings = json.load(f)
with open('../../models/projects/project_bias_embeddings.json') as f:
    movie_bias_embeddings = json.load(f)

In [66]:
def predict(user_id, project_id):
    user_embedding = np.array(user_embeddings[user_id])
    project_embedding = np.array(project_embeddings[project_id])

    # Passing user embedding and movie embedding through the concat layer
    concatenated_embeddings = model.get_layer('concat')([user_embedding, project_embedding])

    # Passing the concatenated embeddings through the dense layers
    x = model.get_layer('dense1')(concatenated_embeddings)
    x = model.get_layer('dense2')(x)

    user_bias_embedding = np.array(user_bias_embeddings[user_id])
    project_bias_embedding = np.array(project_bias_embeddings[project_id])

    user_bias_embedding = user_bias_embedding[:, tf.newaxis]
    project_bias_embedding = project_bias_embedding[:, tf.newaxis]

    # Combine embeddings, biases, and pass through the output layer
    input_tensors = [x, user_bias_embedding, project_bias_embedding]
    concatenated_features = model.get_layer('combined_features')(input_tensors)
    
    x = model.get_layer('combined_dense1')(concatenated_features)
    x = model.get_layer('combined_dense2')(x)
    
    x = model.get_layer('output')(x)

    predicted_rating = x[0][0]

    return predicted_rating.numpy()

In [67]:
def user_recommendations(user_id):
    user_ratings = df[df['user_id'] == user_id]
    user_ratings = df[df['score'] != 1]
    recommendation = df[~df['project_id'].isin(user_ratings['project_id'])][['project_id']].drop_duplicates()
    recommendation['score_predict'] = recommendation.apply(lambda x: predict(user_id, x['project_id']), axis=1)
    
    final_rec = recommendation.sort_values(by='score_predict', ascending=False)
    return final_rec

In [68]:
rec = user_recommendations('5f5a90e2-6601-480c-bc08-7592e06beacd')
rec.head()

,project_id,score_predict
1,22a9acc5-428a-41d4-b056-9bebe9183afe,0.995137


In [69]:
def test_predictions(user_id):
    user_ratings = df[df['user_id'] == user_id]
    
    user_top_ratings = user_ratings.sort_values(by='score', ascending=False)

    comparision = df[df['project_id'].isin(user_ratings['project_id'])][['project_id']].drop_duplicates()
    comparision['score_predict'] = comparision.apply(lambda x: predict(user_id, x['project_id']), axis=1)
    comparision = user_top_ratings.merge(comparision, on='project_id')

    return comparision

In [71]:
comparision = test_predictions('5f5a90e2-6601-480c-bc08-7592e06beacd')
comparision

,user_id,project_id,score,enc_user_id,enc_project_id,score_predict
0,5f5a90e2-6601-480c-bc08-7592e06beacd,6f6235eb-a751-43f2-8bad-506c884635db,4,1,5,3.519993
1,5f5a90e2-6601-480c-bc08-7592e06beacd,41edbc3e-2636-4806-bd87-4c19f273bb3c,4,1,2,3.524254
2,5f5a90e2-6601-480c-bc08-7592e06beacd,45ba1903-cafe-4e57-b7ec-1de402d32cc1,1,1,3,2.018772
3,5f5a90e2-6601-480c-bc08-7592e06beacd,22a9acc5-428a-41d4-b056-9bebe9183afe,1,1,0,0.995137
